In [1]:
import os

In [2]:
%pwd

'd:\\ML-project\\Brain-Tumor\\End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML-project\\Brain-Tumor\\End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    testing_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            testing_data="artifacts/data_ingestion/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [10]:
from urllib.parse import urlparse
import pandas as pd

In [43]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def get_files_df(self):
        self.df = pd.read_csv(os.path.join(self.config.testing_data, "Brain Tumor.csv"))
        self.df = self.df[['Image', 'Class']]
        self.df['Image'] += ".jpg"

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255.0,
            validation_split=0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_dataframe(
            dataframe=self.df,
            directory=os.path.join(self.config.testing_data , "Brain Tumor/"),
            x_col = "Image",
            y_col = "Class",
            subset="validation",
            class_mode='raw',
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.pred = model.predict(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [44]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.get_files_df()
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-08-08 17:11:02,793: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-08 17:11:02,797: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-08 17:11:02,798: INFO: common: directory: artifacts created successfully]
Found 1128 validated image filenames.
141/141 [==============================] - 18s 125ms/step


TypeError: Object of type ndarray is not JSON serializable

In [41]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model("my_best_model.epoch60-Loss0.0802.hdf5")
def predict(filename):
        imagename = filename
        test_image = image.load_img(imagename, target_size = (240,240))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image, verbose = 0)[0]
        # print(result)
        return result


In [42]:
i = 0
res = []
for (root,dirs,files) in os.walk('Test_seggregated/', topdown=True):
        print (root)
        print (dirs)
        print (files)
        for file in files:
            if(i>0): print(predict(root+"/"+file))
            i+=1
        print ('--------------------------------')
        

Test_seggregated/
['Normal', 'Tumor']
[]
--------------------------------
Test_seggregated/Normal
[]
['Image627.jpg', 'Image628.jpg', 'Image631.jpg', 'Image632.jpg', 'Image643.jpg', 'Image644.jpg', 'Image645.jpg', 'Image646.jpg']
[0.9997348]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.2856106e-08]
--------------------------------
Test_seggregated/Tumor
[]
['Image3666.jpg', 'Image3667.jpg', 'Image3669.jpg', 'Image3670.jpg', 'Image3674.jpg', 'Image3675.jpg', 'Image3676.jpg', 'Image3679.jpg']
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
--------------------------------


In [57]:
np.unique(np.array(res), return_counts=True)

(array([0, 1], dtype=int64), array([ 344, 1726], dtype=int64))